# Create the Sales Database from Scratch

Warning: this script will drop the tables and any data or modifications made to them.  Please be 100% sure you are wanting to do this!

In [1]:
import pandas as pd
import psycopg2

In [2]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [3]:
cursor = connection.cursor()

## Drop the tables if they exist

In [4]:
#
# drop all the tables in the foreign key order
#

connection.rollback()

query = """

drop table if exists line_items;
drop table if exists sales;
drop table if exists products;
drop table if exists customers;
drop table if exists stores;
drop table if exists holidays;
drop table if exists zip_codes;
drop table if exists cities;
drop table if exists states;

"""

cursor.execute(query)

connection.commit()



# Create tables

In [5]:
#
# create all the tables in the foreign key order
#

connection.rollback()

query = """

create table zip_codes (
  zip varchar(5),
  latitude numeric(7,4),
  longitude numeric(7,4),
  city varchar(32),
  state varchar(2),
  population numeric(7),
  area numeric(9,4),
  density numeric(10,2),
  time_zone varchar(32),
  primary key (zip)
);

create table cities (
    city varchar(32),
    state varchar(2),
    latitude numeric(7,4),
    longitude numeric(7,4),
    population numeric(9),
    area numeric(10,4),
    density numeric(10,2),
    time_zone varchar(32),
    primary key (city,state)
);

create table states (
    state varchar(2),
    state_name varchar(32),
    latitude numeric(7,4),
    longitude numeric(7,4),
    population numeric(10),
    area numeric(10,4),
    density numeric(10,2),
    primary key (state)
);

create table holidays (
  holiday_date date,
  description varchar(32),
  closed_flag boolean,
  primary key (holiday_date)
);

create table products (
  product_id numeric(3),
  description varchar(32),
  primary key (product_id)
);

create table stores (
  store_id numeric(6),
  street varchar(32),
  city varchar(32),
  state varchar(2),
  zip varchar(5),
  latitude numeric(7,4),
  longitude numeric(7,4),
  primary key (store_id)
);

create table customers (
  customer_id numeric(6),
  first_name varchar(32),
  last_name varchar(32),
  street varchar(32),
  city varchar(32),
  state varchar(2),
  zip varchar(5),
  closest_store_id numeric(6),
  distance numeric(3),
  primary key (customer_id),
  foreign key (closest_store_id) references stores(store_id)
);

create table sales (
  store_id numeric(6),
  sale_id numeric(8),
  customer_id numeric(6),
  sale_date date,
  total_amount numeric(5),
  primary key (store_id, sale_id),
  foreign key (customer_id) references customers (customer_id)
);

create table line_items (
  store_id numeric(6),
  sale_id numeric(8),
  line_item_id numeric(3),
  product_id numeric(3),
  quantity numeric(3),
  primary key (store_id, sale_id, line_item_id),
  foreign key (product_id) references products (product_id)
);

"""

cursor.execute(query)

connection.commit()

# Load tables

In [6]:
#
# load the csv files into the database tables in foreign key order
#

connection.rollback()

query = """

copy zip_codes
from '/user/how_to_guides/sales_database/data/zip_codes.csv' delimiter ',' NULL '' csv header;

copy cities
from '/user/how_to_guides/sales_database/data/cities.csv' delimiter ',' NULL '' csv header;

copy states
from '/user/how_to_guides/sales_database/data/states.csv' delimiter ',' NULL '' csv header;

copy holidays
from '/user/how_to_guides/sales_database/data/holidays.csv' delimiter ',' NULL '' csv header;

copy products
from '/user/how_to_guides/sales_database/data/products.csv' delimiter ',' NULL '' csv header;

copy stores
from '/user/how_to_guides/sales_database/data/stores.csv' delimiter ',' NULL '' csv header;

copy customers
from '/user/how_to_guides/sales_database/data/customers.csv' delimiter ',' NULL '' csv header;

copy sales
from '/user/how_to_guides/sales_database/data/sales.csv' delimiter ',' NULL '' csv header;

copy line_items
from '/user/how_to_guides/sales_database/data/line_items.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()